### What & why:

In [our recent work](link), we calculated the solvent effects on the NMR shielding constant of transition metal nuclei ($\sigma$) in multiple complexes, using two variants of the Frozen Density Embedding (FDE) method:

* canonical FDE with the density of the solvent kept fixed (denoted as `fde0`)
* FDE with the freeze-thaw procedure, in which the densities of the solute and solvent are relaxed in each other presence (referred to as `fdeN`)

In addition, we calculated the NMR shielding constants of these nuclei in:

* isolated solute molecules in their vacuum geometry (`vac`)
* isolated solute molecules in their relaxed geometries - as if they were parts of a complex (`isol`)
* full solvated complexes (`super`)

The properties of interest are denoted as $\sigma_{fde0}$, $\sigma_{fdeN}$, $\sigma_{vac}$, $\sigma_{isol}$, $\sigma_{super}$, respectively.

In this notebook I show how to do interactive plots in [Altair](https://altair-viz.github.io/index.html) that allow to use different plots or views to inspect this data.



#### Data description:

* *df_dirac* and *df_adf* are dataframes that collect the data obtained from calculations with DIRAC (with the DC Hamiltonian) and ADF (with the SO-ZORA Hamiltonian). The data is read from from dirac_data.csv and adf_data.csv files.

* some definitions used in plots:

    * useful to discuss solvent effects estimated by a given model:
        * $\delta_{isol} = \sigma_{isol} - \sigma_{vac}$
        * $\delta_{fde0} = \sigma_{fde0} - \sigma_{vac}$
        * $\delta_{fdeN} = \sigma_{fdeN} - \sigma_{vac}$
        * $\delta_{super} = \sigma_{super} - \sigma_{vac}$    
    
    * useful to discuss contributions to solvent effects:    
        * $\Delta_{isol} = \sigma_{isol} - \sigma_{vac} = \delta_{isol}$
        * $\Delta_{fde0} = \sigma_{fde0} - \sigma_{isol}$
        * $\Delta_{fdeN} = \sigma_{fdeN} - \sigma_{fde0}$
        * $\Delta_{super} = \sigma_{super} - \sigma_{fdeN}$       

    * useful for ....statistics....:
        * $d_{isol}  = (\sigma_{isol}  - \sigma_{super})/|\sigma_{vac}|$
        * $d_{fde0}  = (\sigma_{fde0}  - \sigma_{super})/|\sigma_{vac}|$
        * $d_{fdeN}  = (\sigma_{fdeN}  - \sigma_{super})/|\sigma_{vac}|$
        * $d_{super} = (\sigma_{super} - \sigma_{super})/|\sigma_{vac}|$        

* This notebook uses the actual data discussed in [this paper](link).

In [330]:
import pandas as pd
import altair as alt
import altair_viewer

In [331]:
def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 1000
    display.max_colwidth = 1000
    display.width = None
    display.precision = 2
    
set_pandas_display_options()

In [219]:
def def_molprop():

    mol =  ['Ti',
            'V',
            'Cr',
            'Mn',
            'Fe',
            'Co',
            'Ni',
            'Cu',
            'Zn',
            'Nb',
            'Mo',
            'Tc',
            'Ru',
            'Pd',
            'Ag',
            'Cd',
            'Ta',
            'W',
            'Re',
            'Os',
            'Pt',
            'Hg'
    ]
 
    atomic_number_X={}
    atomic_number_X['Ti']=22
    atomic_number_X['V']=23
    atomic_number_X['Cr']=24
    atomic_number_X['Mn']=25
    atomic_number_X['Fe']=26
    atomic_number_X['Co']=27
    atomic_number_X['Ni']=28
    atomic_number_X['Cu']=29
    atomic_number_X['Zn']=30
    atomic_number_X['Nb']=41
    atomic_number_X['Mo']=42
    atomic_number_X['Tc']=43
    atomic_number_X['Ru']=44
    atomic_number_X['Pd']=46
    atomic_number_X['Ag']=47
    atomic_number_X['Cd']=48
    atomic_number_X['Ta']=73
    atomic_number_X['W']=74
    atomic_number_X['Re']=75
    atomic_number_X['Os']=76
    atomic_number_X['Pt']=78
    atomic_number_X['Hg']=80   
    
    solvent={} 
    solvent['Ti']='12TiCl$_{4}$'
    solvent['V']='6C$_6$H$_6$'
    solvent['Cr']='12H$_2$O'
    solvent['Mn']='12H$_2$O'
    solvent['Fe']='6C$_6$H$_6$'
    solvent['Co']='12H$_2$O'
    solvent['Ni']='6C$_6$H$_6$'
    solvent['Cu']='12CH$_3$CN'
    solvent['Zn']='12H$_2$O'
    solvent['Nb']='12CH$_3$CN'
    solvent['Mo']='12H$_2$O'
    solvent['Tc']='12H$_2$O'
    solvent['Ru']='12H$_2$O'
    solvent['Pd']='12H$_2$O'
    solvent['Ag']='12H$_2$O'
    solvent['Cd']='6Cd(CH$_3$)$_2$'
    solvent['Ta']='12CH$_3$CN'
    solvent['W']='12H$_2$O'
    solvent['Re']='12H$_2$O'
    solvent['Os']='12CCl$_4$'
    solvent['Pt']='12H$_2$O'
    solvent['Hg']='6Hg(CH$_3$)$_2$'

    charge={} 
    charge['Ti']=0
    charge['V']=0
    charge['Cr']=-2
    charge['Mn']=-1
    charge['Fe']=0
    charge['Co']=-3
    charge['Ni']=0
    charge['Cu']=1
    charge['Zn']=2
    charge['Nb']=-1
    charge['Mo']=-2
    charge['Tc']=-1
    charge['Ru']=-4
    charge['Pd']=-2
    charge['Ag']=1
    charge['Cd']=0
    charge['Ta']=-1
    charge['W']=-2
    charge['Re']=-1
    charge['Os']=0
    charge['Pt']=-2
    charge['Hg']=0
    
    row={} 
    row['Ti']=4
    row['V']=0
    row['Cr']=4
    row['Mn']=4
    row['Fe']=4
    row['Co']=4
    row['Ni']=4
    row['Cu']=4
    row['Zn']=4
    row['Nb']=5
    row['Mo']=5
    row['Tc']=5
    row['Ru']=5
    row['Pd']=5
    row['Ag']=5
    row['Cd']=5
    row['Ta']=6
    row['W']=6
    row['Re']=6
    row['Os']=6
    row['Pt']=6
    row['Hg']=6
      
    order_where=['DC','SO-ZORA']

In [220]:
def prep_adf(df):
   
    df_se = df[['mol']].copy()
    df_se['x_labels'] = df_se.mol
    df_se['where'] = 'SO-ZORA'
    df_se['solveff']  = df['supermolecule']-df['isolated_vac']  
    df_se = df_se.drop(['mol'], axis=1)
    df_se = df_se.melt(id_vars =['x_labels', 'where'])     
    
    df_delta = df[['mol']].copy()
    df_delta['x_labels'] = df.mol
    df_delta['where'] = 'SO-ZORA' 
    df_delta['delta1']  = df['isolated_supergeom']-df['isolated_vac']
    df_delta['delta2']  = df['fde']-df['isolated_vac']
    df_delta['delta3']  = df['fnt']-df['isolated_vac']    
    df_delta['delta4']  = df['supermolecule']-df['isolated_vac']     
    df_delta=df_delta.drop(['mol'], axis=1)
    df_delta = df_delta.melt(id_vars =['x_labels', 'where']) 
    
    df_Delta = df[['mol']].copy()
    df_Delta['x_labels'] = df.mol
    df_Delta['where'] = 'SO-ZORA'
    df_Delta['Delta1']  = df['isolated_supergeom']-df['isolated_vac']
    df_Delta['Delta2']  = df['fde']-df['isolated_supergeom']
    df_Delta['Delta3']  = df['fnt']-df['fde']    
    df_Delta['Delta4']  = df['supermolecule']-df['fnt']   
    df_Delta=df_Delta.drop(['mol'], axis=1)
    df_Delta = df_Delta.melt(id_vars =['x_labels', 'where']) 

    df_d = df[['mol']].copy()
    df_d['x_labels'] = df.mol
    df_d['where'] = 'SO-ZORA'    
    df_d['d1']      = (df['isolated_supergeom']-df['supermolecule'])/df['isolated_vac']
    df_d['d2']      = (df['fde']-df['supermolecule'])/df['isolated_vac']
    df_d['d3']      = (df['fnt']-df['supermolecule'])/df['isolated_vac']
    df_d['d4']      = (df['supermolecule']-df['supermolecule'])/df['isolated_vac']
    df_d=df_d.drop(['mol'], axis=1)
    df_d = df_d.melt(id_vars =['x_labels', 'where'])
    
    df_all = pd.concat([df_se, df_delta, df_Delta, df_d],axis=1)
    
    return df_all, df_se, df_delta, df_Delta, df_d


def prep_dirac(df):
    
    df_se = df[['mol']].copy()
    df_se['x_labels'] = df_se.mol
    df_se['where'] = 'DC'
    df_se['solveff']  = df['supermolecule']-df['isolated_vac']
    df_se=df_se.drop(['mol'], axis=1)
    df_se = df_se.melt(id_vars =['x_labels', 'where'])

    df_nose = df[['mol']].copy()
    df_nose['x_labels'] = df.mol
    df_nose['where'] = 'DC'
    df_nose['delta1']  = df['isolated_supergeom_supergrid']-df['isolated_vac']
    df_nose['delta2']  = df['fde_vw11']-df['isolated_vac']
    df_nose['delta3']  = df['fnt_vw11']-df['isolated_vac']    
    df_nose['delta4']  = df['supermolecule']-df['isolated_vac']     
    df_nose['Delta1']  = df['isolated_supergeom_supergrid']-df['isolated_vac']
    df_nose['Delta2']  = df['fde_vw11']-df['isolated_supergeom_supergrid']
    df_nose['Delta3']  = df['fnt_vw11']-df['fde_vw11']    
    df_nose['Delta4']  = df['supermolecule']-df['fnt_vw11'] 
    df_nose['d1']      = (df['isolated_supergeom_supergrid']-df['supermolecule'])/df['isolated_vac']
    df_nose['d2']      = (df['fde_vw11']-df['supermolecule'])/df['isolated_vac']
    df_nose['d3']      = (df['fnt_vw11']-df['supermolecule'])/df['isolated_vac']
    df_nose['d4']      = (df['supermolecule']-df['supermolecule'])/df['isolated_vac']
    df_nose=df_nose.drop(['mol'], axis=1)
    df_nose = df_nose.melt(id_vars =['x_labels', 'where'])

    return df_nose, df_se

In [221]:
df1=pd.read_csv('dirac_data.csv')
df2=pd.read_csv('adf_data.csv')

Let's have a look at the dataframe (DIRAC data):

In [ ]:
df1

And at the second dataframe (ADF data):

In [ ]:
df2

In [ ]:
#df_dirac_se, df_dirac_delta, df_dirac_Delta, df_dirac_d = prep_dirac(df1)

In [252]:
def def_molprop():

    mol =  ['Ti',
            'V',
            'Cr',
            'Mn',
            'Fe',
            'Co',
            'Ni',
            'Cu',
            'Zn',
            'Nb',
            'Mo',
            'Tc',
            'Ru',
            'Pd',
            'Ag',
            'Cd',
            'Ta',
            'W',
            'Re',
            'Os',
            'Pt',
            'Hg'
    ]
 
    atomic_number_X={}
    atomic_number_X['Ti']=22
    atomic_number_X['V']=23
    atomic_number_X['Cr']=24
    atomic_number_X['Mn']=25
    atomic_number_X['Fe']=26
    atomic_number_X['Co']=27
    atomic_number_X['Ni']=28
    atomic_number_X['Cu']=29
    atomic_number_X['Zn']=30
    atomic_number_X['Nb']=41
    atomic_number_X['Mo']=42
    atomic_number_X['Tc']=43
    atomic_number_X['Ru']=44
    atomic_number_X['Pd']=46
    atomic_number_X['Ag']=47
    atomic_number_X['Cd']=48
    atomic_number_X['Ta']=73
    atomic_number_X['W']=74
    atomic_number_X['Re']=75
    atomic_number_X['Os']=76
    atomic_number_X['Pt']=78
    atomic_number_X['Hg']=80     
       
    solvent={} 
    solvent['Ti']='12TiCl₄'
    solvent['V']='6C₆H₆'
    solvent['Cr']='12H₂O'
    solvent['Mn']='12H₂O'
    solvent['Fe']='6C₆H₆'
    solvent['Co']='12H₂O'
    solvent['Ni']='6C₆H₆'
    solvent['Cu']='12CH₃CN'
    solvent['Zn']='12H₂O'
    solvent['Nb']='12CH₃CN'
    solvent['Mo']='12H₂O'
    solvent['Tc']='12H₂O'
    solvent['Ru']='12H₂O'
    solvent['Pd']='12H₂O'
    solvent['Ag']='12H₂O'
    solvent['Cd']='6Cd(CH₃)₂'
    solvent['Ta']='12CH₃CN'
    solvent['W']='12H₂O'
    solvent['Re']='12H₂O'
    solvent['Os']='12CCl₄'
    solvent['Pt']='12H₂O'
    solvent['Hg']='6Hg(CH₃)₂'
    
    cplx={} 
    cplx['Ti']='TiCl₄ + 12TiCl₄'
    cplx['V']='VOCl₃ + 6C₆H₆'
    cplx['Cr']='CrO₄²⁻ + 12H₂O'
    cplx['Mn']='MnO₄⁻ + 12H₂O'
    cplx['Fe']='Fe(CO)₅ + 6C₆H₆'
    cplx['Co']='Co(CN)₆³⁻ + 12H₂O'
    cplx['Ni']='Ni(CO)₄ + 6C₆H₆'
    cplx['Cu']='Cu(CH₃CN)₄⁺ + 12CH₃CN'
    cplx['Zn']='Zn(H₂O)₆²⁺ + 12H₂O'
    cplx['Nb']='NbCl₆⁻ + 12CH₃CN'
    cplx['Mo']='MoO₄²⁻ + 12H₂O'
    cplx['Tc']='TcO₄⁻ + 12H₂O'
    cplx['Ru']='Ru(CN)₆⁴⁻ + 12H₂O'
    cplx['Pd']='PdCl₆²⁻ + 12H₂O'
    cplx['Ag']='Ag(H₂O)₄⁺ + 12H₂O'
    cplx['Cd']='Cd(CH₃)₂ + 6Cd(CH₃)₂'
    cplx['Ta']='TaCl₆⁻ + 12CH₃CN'
    cplx['W']='WO₄²⁻ + 12H₂O'
    cplx['Re']='ReO₄⁻ + 12H₂O'
    cplx['Os']='OsO₄ + 12CCl₄'
    cplx['Pt']='PtCl₆²⁻ + 12H₂O'
    cplx['Hg']='Hg(CH₃)₂ + 6Hg(CH₃)₂'    

    charge={} 
    charge['Ti']=0
    charge['V']=0
    charge['Cr']=-2
    charge['Mn']=-1
    charge['Fe']=0
    charge['Co']=-3
    charge['Ni']=0
    charge['Cu']=1
    charge['Zn']=2
    charge['Nb']=-1
    charge['Mo']=-2
    charge['Tc']=-1
    charge['Ru']=-4
    charge['Pd']=-2
    charge['Ag']=1
    charge['Cd']=0
    charge['Ta']=-1
    charge['W']=-2
    charge['Re']=-1
    charge['Os']=0
    charge['Pt']=-2
    charge['Hg']=0
    
    row={} 
    row['Ti']=4
    row['V']=4
    row['Cr']=4
    row['Mn']=4
    row['Fe']=4
    row['Co']=4
    row['Ni']=4
    row['Cu']=4
    row['Zn']=4
    row['Nb']=5
    row['Mo']=5
    row['Tc']=5
    row['Ru']=5
    row['Pd']=5
    row['Ag']=5
    row['Cd']=5
    row['Ta']=6
    row['W']=6
    row['Re']=6
    row['Os']=6
    row['Pt']=6
    row['Hg']=6
      
    order_where=['DC','SO-ZORA']
    return atomic_number_X, cplx, solvent, charge, row

def prep_adf(df):
    
    an,cplx,solvent,charge,row=def_molprop()
   
    df_se = df[['mol']].copy()
    df_se['x_labels'] = df_se.mol
    df_se['where'] = 'SO-ZORA'
    df_se['solveff']  = df['supermolecule']-df['isolated_vac']
    df_se['row'] = df_se['mol'].map(row)
    df_se['charge'] = df_se['mol'].map(charge)
    df_se['solvent'] = df_se['mol'].map(solvent)
    df_se['complex'] = df_se['mol'].map(cplx)
    #df_se = df_se.drop(['mol'], axis=1)
    #df_se = df_se.melt(id_vars =['x_labels', 'where'])     
    
    df_delta = df[['mol']].copy()
    #df_delta['x_labels'] = df.mol
    #df_delta['where'] = 'SO-ZORA' 
    df_delta['delta1']  = df['isolated_supergeom']-df['isolated_vac']
    df_delta['delta2']  = df['fde']-df['isolated_vac']
    df_delta['delta3']  = df['fnt']-df['isolated_vac']    
    df_delta['delta4']  = df['supermolecule']-df['isolated_vac']     
    df_delta=df_delta.drop(['mol'], axis=1)
    #df_delta = df_delta.melt(id_vars =['x_labels', 'where']) 
    
    df_Delta = df[['mol']].copy()
    #df_Delta['x_labels'] = df.mol
    #df_Delta['where'] = 'SO-ZORA'
    df_Delta['Delta1']  = df['isolated_supergeom']-df['isolated_vac']
    df_Delta['Delta2']  = df['fde']-df['isolated_supergeom']
    df_Delta['Delta3']  = df['fnt']-df['fde']    
    df_Delta['Delta4']  = df['supermolecule']-df['fnt']   
    df_Delta=df_Delta.drop(['mol'], axis=1)
    #df_Delta = df_Delta.melt(id_vars =['x_labels', 'where']) 

    df_d = df[['mol']].copy()
    #df_d['x_labels'] = df.mol
    #df_d['where'] = 'SO-ZORA'    
    df_d['d1']      = (df['isolated_supergeom']-df['supermolecule'])/df['isolated_vac']
    df_d['d2']      = (df['fde']-df['supermolecule'])/df['isolated_vac']
    df_d['d3']      = (df['fnt']-df['supermolecule'])/df['isolated_vac']
    df_d['d4']      = (df['supermolecule']-df['supermolecule'])/df['isolated_vac']
    df_d=df_d.drop(['mol'], axis=1)
    #df_d = df_d.melt(id_vars =['x_labels', 'where'])
    
    df_all = pd.concat([df_se, df_delta, df_Delta, df_d],axis=1,sort=False)
    
    return df_all, df_se, df_delta, df_Delta, df_d


In [253]:

df_adf_all, df_adf_se, df_adf_delta, df_adf_Delta, df_adf_d = prep_adf(df2)

In [242]:
df_adf_all

,mol,x_labels,where,solveff,row,charge,solvent,complex,delta1,delta2,delta3,delta4,Delta1,Delta2,Delta3,Delta4,d1,d2,d3,d4
0,Ti,Ti,SO-ZORA,4.30,4,0,12TiCl₄,TiCl₄+12TiCl₄,22.18,21.68,21.85,4.30,22.18,-0.50,0.17,-17.55,-2.18e-02,-2.12e-02,-2.14e-02,-0.0
1,V,V,SO-ZORA,-25.33,4,0,6C₆H₆,VOCl₃+6C₆H₆,2.81,1.02,-0.11,-25.33,2.81,-1.80,-1.12,-25.23,-1.51e-02,-1.42e-02,-1.36e-02,-0.0
2,Cr,Cr,SO-ZORA,63.86,4,-2,12H₂O,CrO₄²⁻+12H₂O,123.04,104.04,76.65,63.86,123.04,-19.01,-27.38,-12.80,-2.17e-02,-1.47e-02,-4.69e-03,-0.0
3,Mn,Mn,SO-ZORA,-7.24,4,-1,12H₂O,MnO₄⁻+12H₂O,35.49,29.14,15.23,-7.24,35.49,-6.35,-13.91,-22.47,-1.09e-02,-9.25e-03,-5.71e-03,-0.0
4,Fe,Fe,SO-ZORA,47.11,4,0,6C₆H₆,Fe(CO)₅+6C₆H₆,67.43,65.41,64.57,47.11,67.43,-2.02,-0.84,-17.46,-8.72e-03,-7.85e-03,-7.49e-03,-0.0
5,Co,Co,SO-ZORA,1097.37,4,-3,12H₂O,Co(CN)₆³⁻+12H₂O,1098.32,1127.60,1124.48,1097.37,1098.32,29.28,-3.12,-27.11,-1.65e-04,-5.23e-03,-4.69e-03,-0.0
6,Ni,Ni,SO-ZORA,-27.54,4,0,6C₆H₆,6C₆H₆,-9.36,-17.95,-21.46,-27.54,-9.36,-8.59,-3.51,-6.09,-1.12e-02,-5.89e-03,-3.73e-03,-0.0
7,Cu,Cu,SO-ZORA,-53.46,4,1,12CH₃CN,12CH₃CN,-23.30,-44.89,-47.87,-53.46,-23.30,-21.59,-2.98,-5.59,6.32e-02,1.80e-02,1.17e-02,0.0
8,Zn,Zn,SO-ZORA,-43.15,4,2,12H₂O,12H₂O,-29.68,-30.29,-30.12,-43.15,-29.68,-0.61,0.17,-13.02,7.03e-03,6.72e-03,6.80e-03,0.0
9,Nb,Nb,SO-ZORA,-19.51,5,-1,12CH₃CN,12CH₃CN,15.89,3.21,0.18,-19.51,15.89,-12.68,-3.03,-19.69,-1.11e-01,-7.15e-02,-6.19e-02,-0.0


In [325]:
df_adf_all_melted=df_adf_all.melt(id_vars =['mol','x_labels', 'where', 'row', 'charge', 'solvent', 'complex'])
df_adf_all_melted_delta=df_adf_all.melt(id_vars =['mol','x_labels', 'where', 'row', 'charge', 'solvent', 'complex',
                                                 'Delta1','Delta2', 'Delta3', 'Delta4',
                                                 'd1', 'd2', 'd3', 'd4',
                                                 'solveff'])
df_adf_all_melted_delta

,mol,x_labels,where,row,charge,solvent,complex,Delta1,Delta2,Delta3,Delta4,d1,d2,d3,d4,solveff,variable,value
0,Ti,Ti,SO-ZORA,4,0,12TiCl₄,TiCl₄ + 12TiCl₄,22.18,-0.50,0.17,-17.55,-2.18e-02,-2.12e-02,-2.14e-02,-0.0,4.30,delta1,22.18
1,V,V,SO-ZORA,4,0,6C₆H₆,VOCl₃ + 6C₆H₆,2.81,-1.80,-1.12,-25.23,-1.51e-02,-1.42e-02,-1.36e-02,-0.0,-25.33,delta1,2.81
2,Cr,Cr,SO-ZORA,4,-2,12H₂O,CrO₄²⁻ + 12H₂O,123.04,-19.01,-27.38,-12.80,-2.17e-02,-1.47e-02,-4.69e-03,-0.0,63.86,delta1,123.04
3,Mn,Mn,SO-ZORA,4,-1,12H₂O,MnO₄⁻ + 12H₂O,35.49,-6.35,-13.91,-22.47,-1.09e-02,-9.25e-03,-5.71e-03,-0.0,-7.24,delta1,35.49
4,Fe,Fe,SO-ZORA,4,0,6C₆H₆,Fe(CO)₅ + 6C₆H₆,67.43,-2.02,-0.84,-17.46,-8.72e-03,-7.85e-03,-7.49e-03,-0.0,47.11,delta1,67.43
5,Co,Co,SO-ZORA,4,-3,12H₂O,Co(CN)₆³⁻ + 12H₂O,1098.32,29.28,-3.12,-27.11,-1.65e-04,-5.23e-03,-4.69e-03,-0.0,1097.37,delta1,1098.32
6,Ni,Ni,SO-ZORA,4,0,6C₆H₆,Ni(CO)₄ + 6C₆H₆,-9.36,-8.59,-3.51,-6.09,-1.12e-02,-5.89e-03,-3.73e-03,-0.0,-27.54,delta1,-9.36
7,Cu,Cu,SO-ZORA,4,1,12CH₃CN,Cu(CH₃CN)₄⁺ + 12CH₃CN,-23.30,-21.59,-2.98,-5.59,6.32e-02,1.80e-02,1.17e-02,0.0,-53.46,delta1,-23.30
8,Zn,Zn,SO-ZORA,4,2,12H₂O,Zn(H₂O)₆²⁺ + 12H₂O,-29.68,-0.61,0.17,-13.02,7.03e-03,6.72e-03,6.80e-03,0.0,-43.15,delta1,-29.68
9,Nb,Nb,SO-ZORA,5,-1,12CH₃CN,NbCl₆⁻ + 12CH₃CN,15.89,-12.68,-3.03,-19.69,-1.11e-01,-7.15e-02,-6.19e-02,-0.0,-19.51,delta1,15.89


## Total solvent effects:


In [206]:
scatter = alt.Chart(df_adf_se).mark_point(size=100).encode(
    x='x_labels:O',
    y=alt.Y('solveff:O',
            sort='descending',
            axis=alt.Axis(title="Solvent effects [ppm]", 
                          format=",.0f",
                          #values=list(range(1100,-250,-50)),
                          ),
            #scale=alt.Scale(domain=list(range(1100,-250,-50))),
           ),
    color=alt.Color('charge', type='nominal', scale=alt.Scale(scheme='dark2')),
    shape='solvent:O'
    #shape='row:O'
    #color=alt.condition(brush, 'Origin:N', alt.value('lightgray'))
#).add_selection(
#    brush
)

#text = scatter.mark_text(align="center", baseline="bottom").encode(
#       text=alt.Text('solveff:O', format=",.0f")
#    )

#chart = scatter + text
chart = scatter
chart.show()

Displaying chart at http://localhost:22930/


In [237]:
chart=alt.Chart(df_adf_all).mark_point().encode(
    alt.X(alt.repeat("column"), type='ordinal'),
    alt.Y(alt.repeat("row"), type='ordinal',
          sort='descending',
          axis=alt.Axis(format=",.0f")
         ),
    color=alt.Color('charge', type='nominal', scale=alt.Scale(scheme='dark2'))
).properties(
    width=200,
    height=200
).repeat(
    row=['solveff', 'delta3'],
    column=['mol', 'solvent']
).interactive()

chart

alt.RepeatChart(...)

In [285]:
hover = alt.selection_single(on='mouseover', nearest=True, empty='none')

base = alt.Chart(df_adf_all).encode(
    x=alt.X('mol:O',
            axis=alt.Axis(title="X",labelAngle=0)
           ),
    y=alt.Y('solveff',
            axis=alt.Axis(title="δ(super) [ppm]", 
                          format=",.0f")
           ),
    color=alt.condition(hover, 'solvent:N', alt.value('lightgray'))
).properties(
    width=220,
    height=190,
)

points = base.mark_point().add_selection(
    hover
)

text = base.mark_text(dy=-5).encode(
    text = 'complex:N',
    opacity = alt.condition(hover, alt.value(1), alt.value(0))
)

alt.layer(points, text).facet(
    facet=alt.Facet('row:N', title='row(X)'),
    spacing=-60,
    title='Solvent effects on σ(X) [ppm]'
)


alt.FacetChart(...)

In [307]:
hover = alt.selection_single(on='mouseover', nearest=True, empty='none')

def choose_y(ydata):
    base = alt.Chart(df_adf_all).encode(
    x=alt.X('mol:O',
            axis=alt.Axis(title="X",labelAngle=0)
           ),
    y=alt.Y(ydata,
            axis=alt.Axis(title="δ(fdeN) [ppm]", 
                          format=",.0f")
           ),
    color=alt.condition(hover, 'solvent:N', alt.value('lightgray'))
    ).properties(
        width=220,
        height=190,
    )

    points = base.mark_point().add_selection(
        hover
    )

    text = base.mark_text(dy=-5).encode(
    text = 'complex:N',
    opacity = alt.condition(hover, alt.value(1), alt.value(0))
    )
    return points, text

def def_layer(points, text):
    layer=alt.layer(points, text).facet(
        facet=alt.Facet('row:N', title='row(X)'),
        spacing=-60,
        title='Solvent effects on σ(X) [ppm]'
    )
    return layer


def facet_wrap(layers):
    compound_chart = alt.hconcat()
    #for r in range(2,4):  
    for r in layers:
        rowplot = alt.vconcat() #start a new row
        compound_chart &= rowplot # add the entire row of plots as a new row
    return compound_chart
    
delta2_points, delta2_text=choose_y('delta2')
delta3_points, delta3_text=choose_y('delta3')

layer2=def_layer(delta2_points, delta2_text)
layer3=def_layer(delta3_points, delta3_text)

#layers=[layer2,layer3]
#chart=facet_wrap(layers)
#chart
alt.vconcat(layer2, layer3)



alt.VConcatChart(...)

In [329]:
def choose_y(ydata, ydata_title):
    
    hover = alt.selection_single(on='mouseover', nearest=True, empty='none')
    base = alt.Chart(df_adf_all_melted_delta).encode(
    x=alt.X('mol:O',
            axis=alt.Axis(title="X",labelAngle=0)
           ),
    y=alt.Y(ydata,
            axis=alt.Axis(title=ydata_title, 
                          format=",.0f")
           ),
    color=alt.condition(hover, 'solvent:N', alt.value('lightgray'))
    ).properties(
        width=240,
        height=240,
    )

    points = base.mark_point().add_selection(
        hover
    )

    text = base.mark_text(dy=-5).encode(
    #text = 'complex:N',
    text = 'variable:N',
    opacity = alt.condition(hover, alt.value(1), alt.value(0))
    )
    return points, text


def def_layer(points, text):
    layer=alt.layer(points, text).facet(
        facet=alt.Facet('row:N', title='row(X)'),
        spacing=-60,
        title='Solvent effects on σ(X) [ppm]'
    )
    return layer


delta2_points, delta2_text=choose_y('value', 'solvent effects [ppm]')
#delta3_points, delta3_text=choose_y('delta3')

layer2=def_layer(delta2_points, delta2_text)
layer2
#layer3=def_layer(delta3_points, delta3_text)

#alt.vconcat(layer2, layer3)




alt.FacetChart(...)

## Towards one-page interactive summarization

First, let's try to combine what we did above into one intuitive workflow. This can be presented to the readers of ... and to invite them to explore the data on their own:

Let's now try to assemble what we did in a one page summary. I used a 1-page business plan as an inspiration ([link](https://leanstack.com/leancanvas))

In [ ]:
# Create dataframe
data = [[7, 10, 'Alex', 'Smith'],
        [12, 20, 'Bob', 'Jones'],
        [10, 30, 'Clive', 'Smith'],
        [42, 40,  'Alex', 'Johnson']]
df = pd.DataFrame(data,columns=['Favorite number', 'Age', 'First name', 'Last name'])

# Create selections
selection_first_name = alt.selection_multi(fields=['First name'], empty='none')
selection_last_name = alt.selection_multi(fields=['Last name'], empty='none')

# Create interactive scatter plot
scatter = alt.Chart(df).mark_point(size=100).encode(
    x='Favorite number:Q',
    y='Age:Q',
    color=alt.condition(selection_first_name & selection_last_name,
                        alt.Color('First name:N', legend=None),
                        alt.value('lightgray') ),
    shape=alt.Shape('Last name:N', legend=None),
    tooltip=['First name', 'Last name']
).add_selection(
    alt.selection_interval(bind='scales')
)

# Create interactive model name legend
legend_first_name = alt.Chart(df).mark_point(size=100).encode(
    y=alt.Y('First name:N', axis=alt.Axis(orient='right')),
    color=alt.condition(selection_first_name,
                        alt.Color('First name:N', legend=None),
                        alt.value('lightgray') ),
).add_selection(
    selection_first_name

)

# Create interactive model name legend
legend_last_name = alt.Chart(df).mark_point(size=100).encode(
    y=alt.Y('Last name:N', axis=alt.Axis(orient='right')),
    shape=alt.Shape('Last name:N', legend=None),
    color=alt.condition(selection_last_name,
                        alt.value('black'),
                        alt.value('lightgray') ),
).add_selection(
    selection_last_name
)

# Combine plotting elements
chart = scatter | legend_first_name | legend_last_name
#chart.save('test.html')
chart.show()

In [ ]:
df_plot = pd.concat([df1.set_index('mol'),
                     df2.set_index('mol')]).reset_index()
df_plot_te = pd.concat([df1_te.set_index('mol'),
                     df2_te.set_index('mol')]).reset_index()

In [ ]:
df_plot['variable'].replace({'delta1': '\u0394'+'(1)',
                             'delta2': '\u0394'+'(2)',
                             'delta3': '\u0394'+'(3)',
                             'delta4': '\u0394'+'(4)'
                            },inplace=True)

In [ ]:
df_plot_all = pd.merge(df_plot, df_plot_te, on=['mol','name'])

In [ ]:
order_mol=['Cr', 'Mn', 'Co', 'Zn', 'Mo', 'Tc', 'Ru', 'Pd', 'Ag', 'W', 'Re', 'Pt']
order_where=['set1','set2']


bars=alt.Chart(df_plot_all).mark_bar(size=15).encode(     

    # which field to group columns on
    x=alt.X('name:O',
            axis=alt.Axis(grid=True,labelFontSize=8),
            sort=order_where,
            title=None),

    # which field to use as Y values and how to calculate
    y=alt.Y('value_x:Q',
            axis=alt.Axis(grid=True,title=None)),

    # which field to color by & legend
    color=alt.Color('variable_x',
                    scale=alt.Scale(range=['#4381d1', '#47c488', '#ff6f69']),
                    legend=alt.Legend(title="Contributions",
                                      orient="right",
                                      direction="horizontal",
                                      offset=-200,
                                      titleFontSize=16,
                                      labelFontSize=14)),
                   
    # how to order the data on bars
    order=alt.Order('variable_x:Q', sort='ascending'))


# use separate marks for the 'total effect'
rules = alt.Chart(df_plot_all).mark_tick(color='black', 
                                         thickness=1.5,
                                         size=15
                                        ).encode(x=alt.X('name:O',axis=alt.Axis(grid=True,title=None)),
                                                 y=alt.Y('value_y:Q',axis=alt.Axis(grid=True,title=None)))


# combine all together
alt.layer(bars,rules).properties(height=450,width=50).facet(
   column=alt.Column('mol',
                     sort=order_mol,
                     header=alt.Header(title='Contributions to solvent shifts on NMR shieldings',
                                       orient='bottom',
                                       titleFontSize=20,
                                       labelFontSize=14,
                                       labelBaseline='line-top',
                                       labelAlign='center',
                                       labelAnchor='middle'))).resolve_scale(x='independent').configure_view(strokeOpacity=0)

#### Final note

The total effect is additionally marked by horizontal black lines.
It would be better to add these horizontal black lines to the legend, but from what I saw it is not straightfoward to do at this point.

    
#### Thanks:

Scripts used in this notebook are a combination of advice found (mostly) on stackoverlow, which I lost track of... So big thanks to all Altair experts out there!